In [1]:
import numpy as np
import elasticity as ela
from dolfinx import fem, mesh, io
import ufl

In [2]:
elpars = ela.ElasticPars(nel=(50,50,50)) # nel = (50,50,50)) 

In [3]:
fea = ela.Elasticity(elpars)

3 dimensional problem, and mesh size of 50 x 50 x 50 is created


In [4]:
fea.set_boundary_condition()

currently supports left clamp problem only
currently supports point load only


In [5]:
# setup density
density = fem.Function(fea.D0) # if C1, vector per node: power is not defined 
density.x.array[:] = 1.0

In [6]:
petsc_options={"ksp_type": "preonly","pc_type": "lu","pc_factor_mat_solver_type": "mumps"}
fea.setup_problem(density=density, penal=3.)
fea.solve_problem()
print(np.linalg.norm(fea.u_sol.x.array))

2715.049546318871


In [7]:
petsc_options={"ksp_type": "preonly","pc_type": "chol","pc_factor_mat_solver_type": "mumps"}
fea.setup_problem(density=density, penal=3.)
fea.solve_problem()
print(np.linalg.norm(fea.u_sol.x.array))

2715.049546318871


In [8]:
petsc_options={"ksp_type": "preonly","pc_type": "lu","pc_factor_mat_solver_type": "superlu_dist"}
fea.setup_problem(density=density, penal=3.)
fea.solve_problem()
print(np.linalg.norm(fea.u_sol.x.array))

2715.049546318871


In [9]:
petsc_options={"ksp_type": "pcg","pc_type": "ilu"}
fea.setup_problem(density=density, penal=3.)
fea.solve_problem()
print(np.linalg.norm(fea.u_sol.x.array))

2715.049546318871


In [33]:
# testing multigrid solver (ref: https://bitbucket.org/fenics-project/dolfin/src/b55804ecca7d010ff976967af869571b56364975/python/test/unit/la/test_mg_solver.py?at=master#lines-29:87 -- this is not compatible with dolfinx)'
# https://github.com/topopt/TopOpt_in_PETSc/blob/master/LinearElasticity.cc  + PETSC4PY (API:https://www.mcs.anl.gov/petsc/petsc4py-current/docs/apiref/index.html)

# level
nlvls = 4 # smooth_sweeps

# The fine grid solver settings
rtol         = 1.0e-5
atol         = 1.0e-50
dtol         = 1.0e5
restart      = 100
maxitsGlobal = 200

# Coarsegrid solver
coarse_rtol    = 1.0e-8
coarse_atol    = 1.0e-50
coarse_dtol    = 1e5
coarse_maxits  = 30
coarse_restart = 30

ksp_type = "gmres"
pc_type = "mg"

from petsc4py import PETSc

# set up the solver ===============
mg_solver = PETSc.KSP().create(fea.msh.comm)
mg_solver.setType(ksp_type)
mg_solver.setGMRESRestart(restart)
mg_solver.setTolerances(rtol, atol, dtol, maxitsGlobal)
mg_solver.setInitialGuessNonzero(True)
mg_solver.setOperators(fea.problem.A, fea.problem.A)

# the preconditioner 
pc = mg_solver.getPC()
pc.setType(pc_type)

# set solver from options
mg_solver.setFromOptions()

# check if it has changed
pc2= mg_solver.getPC()
assert(pc==pc2)

pcmg_flag = True
# Only if PCMG is used
if pcmg_flag:

    # DMs for grid hierarchy
    da_list = [None] * nlvls
    daclist = [None] * nlvls
    R = None

    # Set 0 to the finest level
    da_nodal = PETSc.DM().Create(fea.msh.comm)
    daclist[0] = da_nodal

    # Coordinates
    xmin, xmax, ymin, ymax, zmin, zmax = (0., fea.elpars.nelx, 0., fea.elpars.nely, 0., fea.elpars.nelz)

    # Set up the coarse meshes
    da_nodal.coarsenHierarchy(nlvls - 1, daclist[1:])
    for k in range(nlvls):
        # NOTE: finest grid is nlevels - 1: PCMG MUST USE THIS ORDER ???
        da_list[k] = daclist[nlvls - 1 - k]
        # THIS SHOULD NOT BE NECESSARY
        # da_list[k].setUniformCoordinates([xmin, xmax, ymin, ymax, zmin, zmax])

    # the PCMG specific options
    pc.setMGLevels(nlvls)
    pc.setMGType(0) # zero if multiplicative
    pc.setMGCycleType(1)
    pc.setMGGalerkinType(0)
    for k in range(1, nlvls):
        R = da_list[k - 1].createInterpolation(da_list[k])
        pc.setMGInterpolation(k, R)
        # R.destroy()

    # # tidy up
    # for k in range(1, nlvls):  # DO NOT DESTROY LEVEL 0
    #     daclist[k].destroy()
    # da_list.destroy()
    # daclist.destroy()


AssertionError: 

In [10]:
# pyvista visualization # TODO:
# import pyvista 
# import dolfinx.plot as plot

# # ref: https://docs.fenicsproject.org/dolfinx/main/python/demos/demo_pyvista.html
# # If environment variable PYVISTA_OFF_SCREEN is set to true save a png
# # otherwise create interactive plot
# if pyvista.OFF_SCREEN:
#     pyvista.start_xvfb(wait=0.1)

# # Set some global options for all plots
# transparent = False
# figsize = 800
# pyvista.rcParams["background"] = [0.5, 0.5, 0.5]
# petsc_options={"ksp_type": "pcgmres","pc_type": "ilu"}
# fea.setup_problem(density=density, penal=3.)
# fea.solve_problem()
# print(np.linalg.norm(fea.u_sol.x.array))

In [11]:
# # Create VTK mesh
# cells, types, x = plot.create_vtk_mesh(fea.msh)
# grid = pyvista.UnstructuredGrid(cells, types, x)

# # # attach the values of u
# # grid.point_data["u"] = fea.u_sol.x.array
# # grid.set_active_scalars("displacement")

# # Create point cloud of vertices, and add the vertex values to the cloud
# print((fea.u_sol.x.array.reshape(x.shape[0], fea.dim)).shape)
# grid.point_data["u"] = fea.u_sol.x.array.reshape(x.shape[0], fea.dim)
# glyphs = grid.glyph(orient="u", factor=0.1)

#  # We would also like to visualize the underlying mesh and obtain
# # that as we have done previously
# num_cells = fea.msh.topology.index_map(fea.msh.topology.dim).size_local
# cell_entities = np.arange(num_cells, dtype=np.int32)
# cells, types, x = plot.create_vtk_mesh(fea.msh, entities=cell_entities)
# org_grid = pyvista.UnstructuredGrid(cells, types, x)


In [12]:
# pyvista.Report(gpu=True)

In [13]:
# We create a pyvista plotter
# plotter = pyvista.Plotter()
# plotter.add_text("Mesh and corresponding vectors",
#                     position="upper_edge", font_size=14, color="black")

# # We add in the glyphs corresponding to the plotter
# plotter.add_mesh(org_grid, color="white", style="wireframe", line_width=5)
# plotter.add_mesh(glyphs)

# # Save as png if we are using a container with no rendering
# if True: #pyvista.OFF_SCREEN:
#     plotter.screenshot("3D_wireframe_with_vectors.png", transparent_background=transparent,
#                         window_size=[figsize, figsize])
# else:
#     plotter.show()


In [14]:
# from petsc4py import PETSc

# # We visualize the data
# plotter = pyvista.Plotter()
# # plotter.add_text("Second-order (P2) discontinuous elements",
#                     # position="upper_edge", font_size=14, color="black")
# sargs = dict(height=0.1, width=0.8, vertical=False, position_x=0.1, position_y=0, color="black")
# plotter.add_mesh(grid, show_edges=False, scalar_bar_args=sargs, line_width=0)
# plotter.add_mesh(org_grid, color="white", style="wireframe", line_width=5)
# plotter.add_mesh(grid.copy(), style="points", point_size=15, render_points_as_spheres=True, line_width=0)
# plotter.view_xy()
# if pyvista.OFF_SCREEN:
#     plotter.screenshot(f"DG_{PETSc.MPI.COMM_WORLD.rank}.png",
#                         transparent_background=transparent, window_size=[figsize, figsize])
# else:
#     plotter.show()

In [34]:
fea.msh